In [9]:
execfile('../src/display_utils.py')
execfile('../src/moldev_utils.py')
%matplotlib inline

In [10]:
summary = pd.read_csv('../output/condition_summary.csv')

In [11]:
colors = {'Blue & Red':'bwr','Blue':'Blues'}

# Well Plate Results

In [12]:
plate_data = pd.read_csv('../output/well_summary.csv')
interact(plot_plate, 
         data = fixed(plate_data),
         parameter = widgets.Select(value = 'Cell Count',
                                    options= get_params(plate_data),
                                    height=250),
         function = plate_data['Function'].unique().tolist(),
         color = colors);

NameError: name 'get_params' is not defined

# How do conditions cluster together?

In [ ]:
data = pipe(summary,
            get_means,
            lambda df: df.set_index('Condition'),
            normalize_columns)

In [ ]:
interact(matrix_tree, data = fixed(data), color = colors);

# Heatmap subsets

In [ ]:
def wrapper(conditions,features,color):
    df = data.loc[list(conditions)][list(features)]
    try: 
        matrix_tree(df,color)
    except:
        print "\n".join(["-"*50, "Please select multiple features and conditions.","-"*50])

subset = interactive(wrapper,
                     conditions = widgets.SelectMultiple(options = data.index.values.tolist(),height = 300),
                     features = widgets.SelectMultiple(options = data.columns.values.tolist(),height = 300),
                     color = colors)
subset

# How do parameters vary with drug dose?

In [ ]:
dose_data = thread_first(summary,
                        lambda df: df[df['Function'] == 'mean'],
                        add_condition_cols)

dose_error = thread_first(summary,
                        lambda df: df[df['Function'] == 'sem'],
                        add_condition_cols)

In [ ]:
interact(dose_plot,
         scale = widgets.ToggleButtons(options=['linear','log']), 
         df = fixed(dose_data),
         err = fixed(dose_error),
         cols = widgets.SelectMultiple(value = ['Cell Count'],
                              options= [col for col in dose_data.columns.values.tolist() if col not in ['Condition','Function','Base','Dose','Unit','Drug']],
                              height=250));

#How are different parameters related?

In [ ]:
from pandas.tools.plotting import scatter_matrix

def scatter(df,cols):
    scatter_matrix(data[list(cols)], alpha=1, figsize=(20, 20), diagonal='kde');
    
interact(scatter, 
         df = fixed(filter_rows(dose_data,'Base','143B')),
         cols = widgets.SelectMultiple(value = ['Cell Count'],
                              options= get_params(dose_data),
                              height=250));

# Parameter vs. Conditions

In [ ]:
raw_cell_data = pd.read_csv('../output/moldev_cleaned.csv')
cell_data = add_condition_cols(raw_cell_data)

In [ ]:
def plot_dist(df,base,parameter, max_shown=3):
    data = df[df['Base'] == base].sort(['Drug','Dose'])
    plt.figure(num=None, figsize=(15, 7), dpi=80, facecolor='w', edgecolor='k')
    sns.boxplot(data[parameter], 
                data['Condition'],
                vert=False,
                order=data.Condition.unique(),
                color=map(colorize,data.Condition.unique()))
    plt.ylabel('Condition',fontsize=20)
    plt.xlabel(parameter,fontsize=20)
    plt.tick_params(labelsize=20)
    mean = data[parameter].mean()
    plt.gca().set_xlim([0 - mean * (max_shown/10),max_shown * mean])

In [ ]:
interactive(plot_dist,
            df = fixed(cell_data), 
            base = cell_data.Base.unique().tolist(), 
            parameter = get_params(cell_data))

In [ ]:
cell_data.head()

def plot_kdes(df,conditions,parameters,bandwidth=0.5,shaded = True,scale='linear'):
    n_rows = int(np.ceil(len(parameters)/3.0))
    plt.figure(figsize=(20,4 * n_rows))
    subs = gridspec.GridSpec(n_rows, 3) 
    plt.subplots_adjust(hspace=0.54,wspace=0.27)

    for parameter,sub in zip(parameters,subs):
        plt.subplot(sub)
        for condition in conditions:
            data = df[df.Condition == condition][parameter]
            sns.distplot(data, hist=False, kde_kws={"shade": shaded, "bw": bandwidth * data.mean()},label=condition)
            plt.legend(loc = 0)
        if scale is not 'linear':
            plt.xscale('log')
        plt.ylabel('Frequency')

In [ ]:
interact(plot_kdes,
         df = fixed(cell_data),
         conditions = widgets.SelectMultiple(options = cell_data.Condition.unique().tolist(),height = 300),
         parameters = widgets.SelectMultiple(options = get_params(cell_data),height = 300),
         bandwidth = (0,0.1,0.02),
         scale = widgets.ToggleButtons(options=['linear','log']));